In [1]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
import astropy.units as u
# from icecream import ic, install

import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

# install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


In [8]:
from astropy.time import Time

foo = ap.Observer(EarthLocation(lat="37:52:41.35", lon="-78:41:34.92", height=566 * u.m), timezone="utc", name="Fan Mountain")
bar = ap.Observer.at_site("APO")
foo.sun_rise_time(Time("2025-05-19 15:00", format="iso"), which="next").iso
# foo, bar

'2025-05-20 10:05:14.356'

In [60]:
conn = connect("../../../../Data/astropaul.db")

session = tlc.ObservingSession(
    ap.Observer(EarthLocation(lat="37:52:41.35", lon="-78:41:34.92", height=566 * u.m), timezone="utc", name="Fan Mountain")
)
session.add_day_range("2025-05-15", "2025-08-21")

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

fan_targets = [
    # target of interest
    "TIC 344541836",
    # wide, bright QEBs
    "TIC 278465736",
    "TIC 470710327",
    "TIC 367448265",
    "TIC 376606423",
    "TIC 317863971",
    "TIC 159089190",
    "TIC 70664819",
    "TIC 286470992",
    "TIC 78568780",
    "TIC 200094011",
]

min_altitude = 35 * u.deg

name = "Fan Mountain SIDE Summer 2025"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Name"].isin(fan_targets))),
    tlc.ancillary_data_from_tess,
    partial(
        tlc.add_observability,
        observing_session=session,
        calc_moon_distance=True,
        observability_threshold=(min_altitude, 80 * u.deg),
    ),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    tlc.add_catalogs,
    tlc.add_lists,
    partial(tlc.add_database_table, table_name="ephemerides"),
    partial(tlc.add_database_table, table_name="dssi_observations"),
    partial(tlc.add_database_table, table_name="speckle_detections"),
    partial(tlc.add_database_table, table_name="pepsi_observations"),
    partial(tlc.add_system_configuration, table_name="DSSI Observations", time_column="Mid JD"),
    partial(tlc.add_system_configuration, table_name="PEPSI Observations", time_column="Mid JD"),
    partial(tlc.add_phase_events, observing_session=session, phase_event_defs=phase_event_defs, event_types=["Eclipse"]),
    # partial(tlc.add_speckle_phase, phase_event_defs=phase_event_defs),
    partial(tlc.add_side_status, phase_event_defs=phase_event_defs),
]
tl = creator.calculate(verbose=False)

# html.render_observing_pages(tl, None, {}, "html")
print(tl.summarize())

C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:314: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list[hours_observable_column] = segment_hours_observable
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:325: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  moon_dist_column = f"{column_name} Min Moon Dist"
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:302: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

Name: Fan Mountain SIDE Summer 2025
Criteria
    lambda df: (df["Target Name"].isin(['TIC 344541836', 'TIC 278465736', 'TIC 470710327', 'TIC 367448265', 'TIC 376606423', 'TIC 317863971', 'TIC 159089190', 'TIC 70664819', 'TIC 286470992', 'TIC 78568780', 'TIC 200094011']))),
    Observability threshold: (<Quantity 35. deg>, <Quantity 80. deg>)
    AltitudeConstraint: {'min': np.float64(35.0), 'max': np.float64(80.0), 'boolean_constraint': True}
    lambda df: (df["Observable Any Night"])),
6 targets:
       6 QuadEB
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 4)
    Observable: (5, 396)
    List: (0, 12)
    Count: (4, 0)
Associated tables:
      99 rows,  2 columns: Lunar Phases
      26 rows,  3 columns: Catalog Membership
      20 rows,  7 columns: Ephemerides
      40 rows, 12 columns: DSSI Observations
      36 rows, 11 columns: Speckle Detections
      34 rows, 17 columns: PEPSI Observations
     732 rows, 10 columns: Phase Events



In [56]:
tl.other_lists["Phase Events"]

,Target Name,System,Member,Orbit Num,Beg JD,Phase,Event,End JD,Beg UTC,End UTC
0,TIC 278465736,A,a,3423,2460814.5112459483,NaN,Eclipse,2460814.6538733337,2025-05-19 00:16:11,2025-05-19 03:41:34
2,TIC 344541836,A,b,870,2460814.5112459483,NaN,Eclipse,2460814.5587836667,2025-05-19 00:16:11,2025-05-19 01:24:38
1,TIC 278465736,A,b,3423,2460814.795606667,0.864324,Eclipse,2460814.920807156,2025-05-19 07:05:40,2025-05-19 10:05:57
3,TIC 470710327,A,b,1854,2460814.799851667,0.896275,Eclipse,2460814.920807156,2025-05-19 07:11:47,2025-05-19 10:05:57
5,TIC 278465736,A,b,3424,2460815.511819137,NaN,Eclipse,2460815.5764833335,2025-05-20 00:17:01,2025-05-20 01:50:08
6,TIC 344541836,B,a,762,2460815.511819137,NaN,Eclipse,2460815.7155546667,2025-05-20 00:17:01,2025-05-20 05:10:23
7,TIC 470710327,A,a,1855,2460815.511819137,NaN,Eclipse,2460815.584849333,2025-05-20 00:17:01,2025-05-20 02:02:10
4,TIC 278465736,A,a,3425,2460815.715626667,0.864324,Eclipse,2460815.8822933338,2025-05-20 05:10:30,2025-05-20 09:10:30
8,TIC 470710327,A,b,1855,2460815.904537667,0.896275,Eclipse,2460815.9203101797,2025-05-20 09:42:32,2025-05-20 10:05:14
11,TIC 470710327,A,a,1856,2460816.5123856575,NaN,Eclipse,2460816.689535333,2025-05-21 00:17:50,2025-05-21 04:32:55


In [61]:
altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
# pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.0)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, f"../../../../Observing Files/{tl.name}")
# pl.categorical_priorities[0]